In [1]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### GTデータの取得

In [2]:
def decimate(data, dt=1):
    """
    時系列データを線形補間し、ダウンサンプリングする。

    Args:
        data (np.ndarray): 2次元配列で、最初の列が時刻、残りの列が対応するデータ値を表す。
        dt (float, optional): ダウンサンプリングのための時間間隔。デフォルトは1。

    Returns:
        np.ndarray: 指定された時間間隔で補間されたダウンサンプリングデータを含む2次元配列。各行はその時間間隔での補間値を表す。
    """

    new_data = []
    pick_time = dt

    for i in range(1, len(data)):
        if data[i, 0] > pick_time:
            x = [data[i - 1, 0], data[i, 0]]
            y = [data[i - 1, :], data[i, :]]

            a, b = np.polyfit(x, y, 1)
            interpolated_value = a * pick_time + b

            new_data.append(interpolated_value)
            pick_time += dt

    new_data = np.array(new_data)
    new_data = new_data.reshape(new_data.shape[0], -1)

    return new_data

In [3]:
def get_gt_data(dataset_list, case_num, num_kw):
    vis_dataset = dataset_list[case_num - 1]

    gt = decimate(pd.read_csv(vis_dataset, skiprows=1).values)
    gt = gt[:, -num_kw:]

    return gt

### 予測値データの取得

In [4]:
def get_pred_data(gt, model_list, predict_dir, case_num, num_kw):

    df_list = []
    df_list.append(gt)

    for model in model_list:
        predict_dataset_path = os.path.join(predict_dir, model, f"case{str(case_num).zfill(4)}.csv")

        # csvファイルから読み込んでkWの次元だけを抽出
        pred_data = pd.read_csv(predict_dataset_path).values
        pred_data = pred_data[:, -num_kw:]

        # in_len部分をgtから拝借して結合
        in_len = gt.shape[0] - pred_data.shape[0]
        pred_data = np.concatenate((gt[:in_len], pred_data), axis=0)
        df_list.append(pred_data)

    df_list = np.stack(df_list, axis=0)

    return df_list, in_len

### プロットする

In [33]:
def create_figure(df_list, case_num, dataset_name, pred_name, pred_unit):
    nrows = 2
    ncols = 2

    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(10*ncols, 7*nrows))

    vis_list = ["Ground Truth", "BaseTransformer", "LSTM", "DeepONet", "DeepOTransformer", "DeepOLSTM"]

    sns.set_palette("bright", len(vis_list))
    sns.set_context("talk") # paper, notebook, talk, poster

    for i in range(nrows):
        for j in range(ncols):
            sns.lineplot(data=pd.DataFrame(df_list[:, :, (nrows-1) * i + (ncols) * j].T, columns=vis_list), ax=axes[i][j])
            sns.set_style("whitegrid", {'grid.linestyle': '--'})

            axes[i][j].set_title(f"{pred_name[(nrows-1) * i + (ncols) * j]} [{pred_unit[(nrows-1) * i + (ncols-1) * j]}]")

    plt.tight_layout()
    plt.legend(loc='best')

    # plt.show()
    os.makedirs(f"/home/dockeruser/code/notebook/{dataset_name}", exist_ok=True)
    plt.savefig(f"/home/dockeruser/code/notebook/{dataset_name}/case{str(case_num).zfill(4)}.png", format="png", dpi=300)

    plt.close()

### パラメータ類

In [6]:
dataset_name = "data_step2"

csv_dir = "/home/dockeruser/code/predict_csvs"
gt_dir = f"/home/dockeruser/dataset/{dataset_name}"
predict_dir = "/home/dockeruser/code/predict_csvs"

model_list = ["bt", "lstm", "don", "dot", "dol"]

if dataset_name == "data_step2":
    num_kw = 4
elif dataset_name == "data_refrig_only":
    num_kw = 5

In [7]:
dataset_list = sorted(glob.glob(os.path.join(gt_dir, "*.csv")))

In [ ]:
case_list = sorted(glob.glob(os.path.join(predict_dir, "bt", "*.csv")))
case_num_list = []
for case_path in case_list:
    file_name = os.path.basename(case_path)  # ファイル名を取得
    number_part = file_name.split("case")[1].split(".")[0]  # "case"と".csv"で分割し、真ん中の部分を抽出

    # 数字をint型に変換
    number = int(number_part)
    case_num_list.append(number)


In [9]:
feature_name = [col.split(".")[0] for col in pd.read_csv(dataset_list[0], skiprows=0, dtype=str).columns]
feature_unit = [col.split(".")[0] for col in pd.read_csv(dataset_list[0], skiprows=1, dtype=str).columns]

pred_name = feature_name[-num_kw:]
pred_unit = feature_unit[-num_kw:]

In [10]:
# まとめて実行
for case_num in case_num_list:
    gt = get_gt_data(dataset_list, case_num, num_kw)
    df_list, in_len = get_pred_data(gt, model_list, predict_dir, case_num, num_kw)
    create_figure(df_list, case_num, dataset_name, pred_name, pred_unit)

In [36]:
# 一つずつ実行
case_num = 56 # 64, 415, 173, 56

gt = get_gt_data(dataset_list, case_num, num_kw)
df_list, in_len = get_pred_data(gt, model_list, predict_dir, case_num, num_kw)
create_figure(df_list, case_num, dataset_name, pred_name, pred_unit)